# Yelp Data Challenge - Data Preprocessing

## Dataset Introduction

[Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge)

The Challenge Dataset:

    4.1M reviews and 947K tips by 1M users for 144K businesses
    1.1M business attributes, e.g., hours, parking availability, ambience.
    Aggregated check-ins over time for each of the 125K businesses
    200,000 pictures from the included businesses

Cities:

    U.K.: Edinburgh
    Germany: Karlsruhe
    Canada: Montreal and Waterloo
    U.S.: Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, Cleveland

Files:

    yelp_academic_dataset_business.json
    yelp_academic_dataset_checkin.json
    yelp_academic_dataset_review.json
    yelp_academic_dataset_tip.json
    yelp_academic_dataset_user.json

Notes on the Dataset

    Each file is composed of a single object type, one json-object per-line.
    Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.



## Read data from file and load to Pandas DataFrame

In [1]:
import json
import pandas as pd

In [2]:
file_business, file_checkin, file_review, file_tip, file_user = [
    'yelp_dataset_challenge_round9/yelp_academic_dataset_business.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_checkin.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_review.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_tip.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_user.json'
]

#### Business Data

In [3]:
with open(file_business) as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [4]:
df_business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,type
0,"227 E Baseline Rd, Ste J2","[BikeParking: True, BusinessAcceptsBitcoin: Fa...",0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]",Tempe,"[Monday 11:0-21:0, Tuesday 11:0-21:0, Wednesda...",0,33.378214,-111.936102,Innovative Vapors,,85283,17,4.5,AZ,business
1,495 S Grand Central Pkwy,"[BusinessAcceptsBitcoin: False, BusinessAccept...",LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser...",Las Vegas,"[Monday 0:0-0:0, Tuesday 0:0-0:0, Wednesday 0:...",1,36.192284,-115.159272,Cut and Taste,,89106,9,5.0,NV,business


In [ ]:
df_business.info()

#### Checkin Data

In [ ]:
# with open(file_checkin) as f:
#     df_checkin = pd.DataFrame(json.loads(line) for line in f)
# df_checkin.head(2)

#### Review Data

In [ ]:
# with open(file_review) as f:
#     df_review = pd.DataFrame(json.loads(line) for line in f)
# df_review.head(2)

#### Tip Data

In [ ]:
# with open(file_tip) as f:
#     df_tip = pd.DataFrame(json.loads(line) for line in f)
# df_tip.head(2)

#### User Data

In [ ]:
# with open(file_user) as f:
#     df_user = pd.DataFrame(json.loads(line) for line in f)
# df_user.head(2)

## Filter data by city and category

#### Create filters/masks

* create filters that selects business 
    * that are located in "Las Vegas"
    * that contains "Restaurants" in their category (You may need to filter null categories first)

In [ ]:
# Create Pandas DataFrame filters
df_business_cate_noNa = df_business.categories.dropna()
df_business_cate_noNa = df_business_cate_noNa.reset_index()

filter_category = list()
for index, line in enumerate(df_business_cate_noNa.categories):
    for word in line:
        if word == "Restaurants":
            filter_category.append(index)
df_business_cate_noNa = df_business_cate_noNa.ix[filter_category]
df_business_cate_noNa = df_business_cate_noNa.set_index('index')
df_filtered1 = pd.merge(df_business_cate_noNa,df_business, how='inner',left_index = True, right_index = True)
df_filtered1 = df_filtered1.drop("categories_x", axis=1)
df_filtered1 = df_filtered1.rename(columns={'categories_y':'categories'})
filter_city = df_filtered1.city == "Las Vegas"

In [ ]:
# Create filtered DataFrame, and name it df_filtered
df_filtered = df_filtered1[filter_city]


#### Keep relevant columns

* only keep some useful columns
    * business_id
    * name
    * categories
    * stars

In [ ]:
selected_features = [u'business_id', u'name', u'categories', u'stars']

In [ ]:
# Make a DataFrame that contains only the abovementioned columns, and name it as df_selected_business
df_selected_business = df_filtered[selected_features]

In [ ]:
# Rename the column name "stars" to "avg_stars" to avoid naming conflicts with review dataset
df_selected_business = df_selected_business.rename(columns={'stars':'avg_stars'})

In [ ]:
# Inspect your DataFrame
df_selected_business

#### Save results to csv files

In [ ]:
# Save to ./data/selected_business.csv for your next task
df_selected_business.to_csv("data/selected_business.csv", index = False, encoding = 'utf-8')

In [ ]:
# Try reload the csv file to check if everything works fine
pd.read_csv("data/selected_business.csv")

### Use the "business_id" column to filter review data

* We want to make a DataFrame that contain and only contain the reviews about the business entities we just obtained

#### Load review dataset

In [ ]:
with open(file_review) as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
df_review.head(2)

#### Prepare dataframes to be joined, - on business_id

In [ ]:
# Prepare the business dataframe and set index to column "business_id", and name it as df_left
df_left = pd.read_csv("data/selected_business.csv")
df_left = df_left.set_index('business_id')

In [ ]:
# Prepare the review dataframe and set index to column "business_id", and name it as df_right
df_right = df_review.set_index('business_id')

#### Join! and reset index

In [ ]:
# Join df_left and df_right. What type of join?
df_join = pd.merge(df_left,df_right,how='inner',left_index = True,right_index = True)


In [ ]:
# You may want to reset the index 
df_join = df_join.reset_index()

#### We further filter data by date, e.g. keep comments from last 2 years

* Otherwise your laptop may crush on memory when running machine learning algorithms
* Purposefully ignoring the reviews made too long time ago

In [ ]:
# Make a filter that selects date after 2015-01-20
filter_date = pd.to_datetime(df_join.date) > "2015-01-20"


In [ ]:
# Filter the joined DataFrame and name it as df_final
df_final = df_join[filter_date]
df_final

#### Take a glance at the final dataset

* Do more EDA here as you like!

In [ ]:
import matplotlib.pyplot as plt

% matplotlib inline
plt.style.use('ggplot')

In [ ]:
# e.g. calculate counts of reviews per business entity, and plot it
df_count = df_final.copy()
df_count["count"]=1
df_counts = df_count.groupby('business_id').sum()
df_counts["count"].plot(title = 'Counts of reviews per business entity',figsize = (24,4))


## Save your preprocessed dataset to csv file

* Respect your laptop's hard work! You don't want to make it run everything again.

In [ ]:
# Save to ./data/last_2_years_restaurant_reviews.csv for your next task
df_final.to_csv("data/last_2_years_restaurant_reviews.csv", index = False, encoding = 'utf-8')